In [3]:
class Persona:
    def __init__(self, nombre, contacto):
        self.nombre = nombre
        self.contacto = contacto
        
    def actualizar_datos(self, nombre=None, contacto=None):
        if nombre:
            self.nombre = nombre
        if contacto:
            self.contacto = contacto

class Cliente(Persona):
    def __init__(self, nombre, contacto):
        super().__init__(nombre, contacto)
        self.historial_pedidos = []
        self.puntos_fidelidad = 0
    
    def realizar_pedido(self, productos, personalizaciones=None):
        if Inventario.verificar_stock(productos):
            nuevo_pedido = Pedido(self, productos, personalizaciones)
            self.historial_pedidos.append(nuevo_pedido)
            self.puntos_fidelidad += len(productos)  # 1 punto por producto
            return nuevo_pedido
        return None
    
    def consultar_historial(self):
        return self.historial_pedidos

class Empleado(Persona):
    def __init__(self, nombre, contacto, rol):
        super().__init__(nombre, contacto)
        self.rol = rol  # mesero, barista, gerente
    
    def actualizar_inventario(self, producto, cantidad):
        return Inventario.actualizar_stock(producto, cantidad)
    
    def preparar_pedido(self, pedido):
        if pedido.estado == "pendiente":
            pedido.estado = "en_preparacion"
            return True
        return False
    
    def entregar_pedido(self, pedido):
        if pedido.estado == "en_preparacion":
            pedido.estado = "entregado"
            return True
        return False

class ProductoBase:
    def __init__(self, nombre, precio, stock_minimo):
        self.nombre = nombre
        self.precio = precio
        self.stock_minimo = stock_minimo

class Bebida(ProductoBase):
    def __init__(self, nombre, precio, stock_minimo, tamaño, tipo):
        super().__init__(nombre, precio, stock_minimo)
        self.tamaño = tamaño  # pequeño, mediano, grande
        self.tipo = tipo      # caliente/fría
        self.opciones_personalizables = {
            "tipo_leche": ["normal", "descremada", "almendra", "soya"],
            "azucar": ["normal", "sin azúcar", "endulzante"],
            "extras": ["crema", "canela", "chocolate"]
        }

class Postre(ProductoBase):
    def __init__(self, nombre, precio, stock_minimo, es_vegano=False, sin_gluten=False):
        super().__init__(nombre, precio, stock_minimo)
        self.es_vegano = es_vegano
        self.sin_gluten = sin_gluten

class Inventario:
    _stock = {}  # Diccionario para mantener el stock de productos
    
    @classmethod
    def actualizar_stock(cls, producto, cantidad):
        if producto not in cls._stock:
            cls._stock[producto] = 0
        cls._stock[producto] += cantidad
        return cls._stock[producto]
    
    @classmethod
    def verificar_stock(cls, productos):
        for producto in productos:
            if producto not in cls._stock or cls._stock[producto] <= producto.stock_minimo:
                return False
        return True
    
    @classmethod
    def descontar_stock(cls, productos):
        if cls.verificar_stock(productos):
            for producto in productos:
                cls._stock[producto] -= 1
            return True
        return False

class Pedido:
    def __init__(self, cliente, productos, personalizaciones=None):
        self.cliente = cliente
        self.productos = productos
        self.personalizaciones = personalizaciones or {}
        self.estado = "pendiente"  # pendiente, en_preparacion, entregado
        self.total = self.calcular_total()
    
    def calcular_total(self):
        total = sum(producto.precio for producto in self.productos)
        # Aplicar promociones si existen
        promociones_aplicables = Promocion.obtener_promociones(self)
        for promocion in promociones_aplicables:
            total = promocion.aplicar_descuento(total)
        return total
    
    def agregar_producto(self, producto, personalizacion=None):
        if Inventario.verificar_stock([producto]):
            self.productos.append(producto)
            if personalizacion:
                self.personalizaciones[producto] = personalizacion
            self.total = self.calcular_total()
            return True
        return False

class Promocion:
    _promociones = []
    
    def __init__(self, nombre, tipo, descuento, condicion):
        self.nombre = nombre
        self.tipo = tipo  # "producto" o "pedido"
        self.descuento = descuento  # porcentaje de descuento
        self.condicion = condicion  # función que evalúa si aplica la promoción
        Promocion._promociones.append(self)
    
    def aplicar_descuento(self, total):
        return total * (1 - self.descuento)
    
    @classmethod
    def obtener_promociones(cls, pedido):
        return [promo for promo in cls._promociones if promo.condicion(pedido)]

#Ejemplo de uso
def crear_promocion_fidelidad():
    def condicion_fidelidad(pedido):
        return pedido.cliente.puntos_fidelidad >= 50
    return Promocion("Cliente Frecuente", "pedido", 0.1, condicion_fidelidad)

##EJMPLO DE FUNCIONAMIENTO

In [4]:
def ejecutar_ejemplo_cafeteria():
    #Crear productos
    print("=== Creando Productos ===")
    cafe_americano = Bebida("Café Americano", 25, 5, "mediano", "caliente")
    cafe_latte = Bebida("Café Latte", 35, 5, "grande", "caliente")
    frappe = Bebida("Frappé", 45, 3, "grande", "fría")
    croissant = Postre("Croissant", 30, 5, es_vegano=False, sin_gluten=False)
    muffin_vegano = Postre("Muffin de Plátano", 28, 3, es_vegano=True, sin_gluten=True)
    
    print("Productos creados:")
    for producto in [cafe_americano, cafe_latte, frappe, croissant, muffin_vegano]:
        print(f"- {producto.nombre}: ${producto.precio}")
    
    #Actualizar inventario
    print("\n=== Actualizando Inventario ===")
    Inventario.actualizar_stock(cafe_americano, 20)
    Inventario.actualizar_stock(cafe_latte, 15)
    Inventario.actualizar_stock(frappe, 10)
    Inventario.actualizar_stock(croissant, 12)
    Inventario.actualizar_stock(muffin_vegano, 8)
    
    print("Stock actual:")
    for producto in Inventario._stock:
        print(f"- {producto.nombre}: {Inventario._stock[producto]} unidades")
    
    #Crear empleados
    print("\n=== Creando Personal ===")
    barista = Empleado("Juan", "juan@cafeteria.com", "barista")
    mesero = Empleado("María", "maria@cafeteria.com", "mesero")
    gerente = Empleado("Carlos", "carlos@cafeteria.com", "gerente")
    
    print("Personal de la cafetería:")
    for empleado in [barista, mesero, gerente]:
        print(f"- {empleado.nombre}: {empleado.rol}")
    
    #Crear clientes
    print("\n=== Registrando Clientes ===")
    cliente1 = Cliente("Ana", "ana@email.com")
    cliente2 = Cliente("Pedro", "pedro@email.com")
    
    #Crear promociones
    print("\n=== Configurando Promociones ===")
    def condicion_fidelidad(pedido):
        return pedido.cliente.puntos_fidelidad >= 10
        
    promocion_fidelidad = Promocion(
        "Cliente Frecuente", 
        "pedido", 
        0.15,  # 15% de descuento
        condicion_fidelidad
    )
    
    print(f"Promoción creada: {promocion_fidelidad.nombre} - {promocion_fidelidad.descuento*100}% descuento")
    
    #Realizar pedidos
    print("\n=== Procesando Pedidos ===")
    #Pedido de Ana
    personalizacion_cafe = {
        "tipo_leche": "almendra",
        "azucar": "sin azúcar",
        "extras": ["canela"]
    }
    
    pedido1 = cliente1.realizar_pedido(
        [cafe_latte, croissant],
        {cafe_latte: personalizacion_cafe}
    )
    
    print(f"\nPedido de {cliente1.nombre}:")
    print("Productos:")
    for producto in pedido1.productos:
        print(f"- {producto.nombre}")
    if cafe_latte in pedido1.personalizaciones:
        print("Personalización del café:")
        for key, value in pedido1.personalizaciones[cafe_latte].items():
            print(f"  {key}: {value}")
    print(f"Total: ${pedido1.total}")
    
    #Procesar pedido
    print("\n=== Preparación y Entrega ===")
    print("Estado inicial:", pedido1.estado)
    
    barista.preparar_pedido(pedido1)
    print("Después de preparación:", pedido1.estado)
    
    barista.entregar_pedido(pedido1)
    print("Después de entrega:", pedido1.estado)
    
    #Verificar puntos de fidelidad
    print("\n=== Programa de Fidelidad ===")
    print(f"Puntos de fidelidad de {cliente1.nombre}: {cliente1.puntos_fidelidad}")
    
    #Consultar historial
    print("\n=== Historial de Pedidos ===")
    historial = cliente1.consultar_historial()
    print(f"Pedidos de {cliente1.nombre}:")
    for pedido in historial:
        print(f"- Pedido con {len(pedido.productos)} productos - Estado: {pedido.estado}")

if __name__ == "__main__":
    ejecutar_ejemplo_cafeteria()

=== Creando Productos ===
Productos creados:
- Café Americano: $25
- Café Latte: $35
- Frappé: $45
- Croissant: $30
- Muffin de Plátano: $28

=== Actualizando Inventario ===
Stock actual:
- Café Americano: 20 unidades
- Café Latte: 15 unidades
- Frappé: 10 unidades
- Croissant: 12 unidades
- Muffin de Plátano: 8 unidades

=== Creando Personal ===
Personal de la cafetería:
- Juan: barista
- María: mesero
- Carlos: gerente

=== Registrando Clientes ===

=== Configurando Promociones ===
Promoción creada: Cliente Frecuente - 15.0% descuento

=== Procesando Pedidos ===

Pedido de Ana:
Productos:
- Café Latte
- Croissant
Personalización del café:
  tipo_leche: almendra
  azucar: sin azúcar
  extras: ['canela']
Total: $65

=== Preparación y Entrega ===
Estado inicial: pendiente
Después de preparación: en_preparacion
Después de entrega: entregado

=== Programa de Fidelidad ===
Puntos de fidelidad de Ana: 2

=== Historial de Pedidos ===
Pedidos de Ana:
- Pedido con 2 productos - Estado: entrega